# 第一模块：离线知识库构建

本 Notebook 实现酒店评论 RAG 系统的离线知识库构建流程：

1. **评论质量评估** - 使用 LLM 对评论进行质量评分
2. **评论分类** - 将评论归类到 5 大类中的 14 小类
3. **类别摘要生成** - 为每个小类生成摘要
4. **反向 Query 生成** - 为每条评论生成 1-3 个问题
5. **向量数据库构建** - 构建 3 个向量数据库（评论库、反向 Query 库、摘要库）
6. **倒排索引构建** - 基于 BM25 算法构建文本检索索引

**前置条件**：
- 清洗后的数据文件：`data/processed/hotel_comments_cleaned.csv`
- 配置文件：`config/categories.json`, `config/stopwords_chinese.txt`
- 环境变量：`DASHSCOPE_API_KEY`, `DASHVECTOR_API_KEY`, `DASHVECTOR_HOTEL_ENDPOINT`

## 环境配置

`pip install dashscope dashvector chromadb`

In [1]:
import os
import re
import json
import time
import nltk
import jieba
import pickle
import pandas as pd
from pathlib import Path
from collections import defaultdict, Counter
from tqdm.notebook import tqdm

# 下载停用词
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# API 客户端
from dashscope import Generation, TextEmbedding
import dashvector
from dashvector import Doc
import chromadb

In [2]:
# 检查环境变量
required_env = {
    "DASHSCOPE_API_KEY": os.getenv("DASHSCOPE_API_KEY"),
    "DASHVECTOR_API_KEY": os.getenv("DASHVECTOR_API_KEY"),
    "DASHVECTOR_HOTEL_ENDPOINT": os.getenv("DASHVECTOR_HOTEL_ENDPOINT"),
}

missing = [k for k, v in required_env.items() if not v]
if missing:
    raise EnvironmentError(f"缺少环境变量: {', '.join(missing)}")

print("环境变量检测成功:")
for key, value in required_env.items():
    print(f"- {key}: {value[:10]}...{value[-4:]}")

环境变量检测成功:
- DASHSCOPE_API_KEY: sk-5540c66...f37a
- DASHVECTOR_API_KEY: sk-W6G93uB...7EDE
- DASHVECTOR_HOTEL_ENDPOINT: vrs-cn-e4k....com


In [3]:
# 项目路径配置
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DATA_DIR = DATA_DIR / "processed"
CONFIG_DIR = PROJECT_ROOT / "config"

# 模型配置
LLM_MODEL = "deepseek-v3.2"
EMBEDDING_MODEL = "text-embedding-v4"
EMBEDDING_DIMENSION = 1024

print(f"项目根目录: {PROJECT_ROOT}")
print(f"LLM 模型: {LLM_MODEL}")
print(f"Emb 模型: {EMBEDDING_MODEL}（维度: {EMBEDDING_DIMENSION}）")

项目根目录: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论
LLM 模型: deepseek-v3.2
Emb 模型: text-embedding-v4（维度: 1024）


## 初始化

In [4]:
# LLM 客户端
class LLMClient:
    """Qwen 客户端封装"""
    def __init__(self, api_key: str, model: str = "qwen-plus", json: bool = False):
        self.api_key = api_key
        self.model = model
        self.json = json
    
    def generate(self, prompt: str, temperature: float = 0.7) -> str:
        """生成文本"""
        response = Generation.call(
            api_key=self.api_key,
            model=self.model,
            prompt=prompt,
            temperature=temperature,
            result_format="message",
            response_format={"type": "json_object"} if self.json else None
        )
        
        if response.status_code == 200:
            return response.output.choices[0].message.content.strip()
        else:
            raise RuntimeError(f"LLM 调用失败: {response.message}")


# Embedding 客户端
class EmbeddingClient:
    """文本嵌入客户端封装"""
    def __init__(self, api_key: str, model: str = "text-embedding-v4", dimension: int = 1024):
        self.api_key = api_key
        self.model = model
        self.dimension = dimension
    
    def embed_batch(self, texts: list[str]) -> list[list[float]]:
        """批量生成 embedding"""
        response = TextEmbedding.call(
            api_key=self.api_key,
            model=self.model,
            input=texts,
            dimension=self.dimension
        )
        
        if response.status_code == 200:
            return [item['embedding'] for item in response.output['embeddings']]
        else:
            raise RuntimeError(f"Embedding 调用失败: {response.message}")

In [5]:
# 初始化 LLM & Embedding 客户端
llm_client = LLMClient(api_key=required_env["DASHSCOPE_API_KEY"], model=LLM_MODEL, json=True)
embedding_client = EmbeddingClient(
    api_key=required_env["DASHSCOPE_API_KEY"],
    model=EMBEDDING_MODEL,
    dimension=EMBEDDING_DIMENSION
)
BATCH_SIZE = 10  # text-embedding-v4 模型支持的最大 batch size

# 初始化向量数据库客户端
dashvector_client = dashvector.Client(
    api_key=required_env["DASHVECTOR_API_KEY"],
    endpoint=required_env["DASHVECTOR_HOTEL_ENDPOINT"]
)

chroma_db_path = DATA_DIR / "chroma_db"
chroma_db_path.mkdir(parents=True, exist_ok=True)
chroma_client = chromadb.PersistentClient(path=str(chroma_db_path))
print(f"ChromaDB Path: {chroma_db_path}")

ChromaDB Path: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\chroma_db


In [6]:
# 加载数据
raw_data_file = PROCESSED_DATA_DIR / "hotel_comments_cleaned.csv"
df_raw = pd.read_csv(raw_data_file, index_col=0)

print(f"原数据共 {len(df_raw)} 条评论")
df_raw.head(1)

原数据共 2542 条评论


,comment,images,score,publish_date,room_type,fuzzy_room_type,travel_type,comment_len,log_comment_len,useful_count,log_useful_count,review_count,log_review_count
_id,,,,,,,,,,,,,
68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,"[ ""https://dimg04.c-ctrip.com/images/0230y1200...",5.0,2025-04-05,红棉大床套房,套房,家庭亲子,320,5.771441,0,0.0,7,2.079442


## 步骤1：评论质量评估

对每条评论进行质量评分（0-10分），过滤低质量评论（≤4分）
- 可以与步骤二、四合并，每条评论只调用一次 LLM
- 可以开启多线程并行调用 API 以节省时间
- 可以设置 checkpoint 保存中间结果，避免因网络波动导致的 API 调用中断

In [7]:
# 质量评估 Prompt
QUALITY_ASSESSMENT_PROMPT = """
你是一位专业的评论质量评估专家,请对以下酒店评论进行质量打分(0-10分)

评分标准:
- 9-10分: 信息丰富,具体详细,有明确的观点和充分的细节支持
- 7-8分: 有一定信息量,提供了较为具体的内容
- 5-6分: 信息量中等,有些笼统但仍有参考价值
- 3-4分: 信息量较少,过于简短或模糊
- 0-2分: 几乎没有实质内容,如只有"好"、"不错"等

评论内容: {comment}

直接返回JSON格式:
{{
    "quality_score": 分数(0-10的整数)
}}
"""

def assess_quality(comment: str, index: int) -> int:
    """评估单条评论质量"""
    prompt = QUALITY_ASSESSMENT_PROMPT.format(comment=comment)
    
    for i in range(3):
        try:
            response = llm_client.generate(prompt, temperature=0.3)
            response = response.replace('```json', '').replace('```', '').strip()
            data = json.loads(response)
            return int(data['quality_score'])
        except Exception as e:
            print(f"索引为 {index} 的评论第 {i+1} 次尝试失败: {e}")
            if i < 2:
                time.sleep(1)
                continue

    print(f"索引为 {index} 的评论评估失败，已返回 -1")
    return -1

In [8]:
# 批量质量评估
quality_results = []

for index, row in tqdm(df_raw.iterrows(), total=len(df_raw), desc="质量评估"):
    score = assess_quality(row['comment'], index)
    quality_results.append(score)
    time.sleep(0.1)  # API 限流保护

df_raw['quality_score'] = quality_results

print(f"平均质量分: {df_raw['quality_score'].mean():.2f}")
print(f"质量分布:\n{df_raw['quality_score'].value_counts().sort_index()}")

质量评估:   0%|          | 0/2542 [00:00<?, ?it/s]

平均质量分: 6.87
质量分布:
quality_score
0       4
1      15
2      45
3     110
4     197
5     181
6     306
7     542
8     589
9     537
10     16
Name: count, dtype: int64


In [9]:
# 查看低质量评论数量
num_raw = len(df_raw)
num_filtered = len(df_raw[df_raw['quality_score'] >= 5])

print(f"原有评论数: {num_raw}")
print(f"保留评论数: {num_filtered} (quality_score >= 5)")
print(f"过滤评论数: {num_raw - num_filtered}")
print(f"保留比例: {num_filtered / num_raw * 100:.1f}%")

原有评论数: 2542
保留评论数: 2171 (quality_score >= 5)
过滤评论数: 371
保留比例: 85.4%


## 步骤2：评论分类

将每条评论归类到 1-3 个小类

In [10]:
# 加载分类体系
with open(CONFIG_DIR / "categories.json", 'r', encoding='utf-8') as f:
    categories_config = json.load(f)

# 提取所需小类名称
my_categories = set()
for cat in categories_config['categories']:
    for subcat in cat['subcategories']:
        my_categories.add(subcat['name'])
print(my_categories, end="\n\n")

# 构建分类提示词的类别字典
def format_categories_for_prompt(config: dict) -> str:
    """格式化分类体系为提示词文本"""
    lines = []
    for cat in config['categories']:
        lines.append(f"【{cat['name']}】")
        for subcat in cat['subcategories']:
            subcategories_str = "、".join(subcat['subcategories'])
            lines.append(f"  - **{subcat['name']}**, 例如: {subcategories_str}等")
    return "\n".join(lines)

categories_text = format_categories_for_prompt(categories_config)
print(categories_text)

{'性价比', '交通便利性', '安静程度', '前台服务', '退房/入住效率', '周边配套', '景观/朝向', '房间设施', '价格合理性', '卫生状况', '公共设施', '餐饮设施', '整体满意度', '客房服务'}

【设施类】
  - **房间设施**, 例如: 床、卫浴、空调、wifi、电视、冰箱、热水等
  - **公共设施**, 例如: 泳池、健身房、停车场、会议室、商务中心等
  - **餐饮设施**, 例如: 早餐、餐厅、酒吧、客房送餐等
【服务类】
  - **前台服务**, 例如: 入住、退房、礼宾、咨询、态度等
  - **客房服务**, 例如: 打扫、整理、送物、维修等
  - **退房/入住效率**, 例如: 速度、流程、等待时间等
【位置类】
  - **交通便利性**, 例如: 地铁、公交、机场、火车站、打车等
  - **周边配套**, 例如: 商场、餐饮、景点、医院、银行等
  - **景观/朝向**, 例如: 江景、园景、城景、无窗等
【价格类】
  - **性价比**, 例如: 值得、划算、超值、合理等
  - **价格合理性**, 例如: 贵、便宜、优惠、折扣等
【体验类】
  - **整体满意度**, 例如: 满意、推荐、舒适、愉快等
  - **安静程度**, 例如: 安静、吵闹、隔音等
  - **卫生状况**, 例如: 干净、卫生、整洁、清洁等


In [11]:
# 分类 Prompt
CLASSIFICATION_PROMPT = """
你是一位专业的评论分类专家,请将以下酒店评论归类到给出的5个大类14个小类中最相关的1-3个小类

可选的分类体系:
{categories}

评论内容: {comment}

请分析评论内容,选择1-3个最相关的小类。若都不是则输出空列表。直接返回JSON格式:
{{
    "categories": ["小类1", "小类2", "小类3"]
}}

输出示例1:
{{
    "categories": ["房间设施", "交通便利性", "安静程度"]
}}

输出示例2:
{{
    "categories": []
}}

注意:
1. categories数组中只能出现以上给出的小类名称(即被加粗的内容,**xxx**),不允许出现大类名称(即被框出的内容,【xxx】),不允许出现多余符号,严禁自行编造未给出的小类名称
2. 若评论中未明确指出具体小类,如仅提到"服务到位",则输出与之相关的小类名称,如["前台服务", "客房服务", "退房/入住效率"],严禁直接输出大类名称["服务类"]
3. 如果评论只涉及1-2个类别或涉及其他类别的内容极少,数组中就只放1-2个元素,不要强行输出3个类别,优先保证类别与评论的相关性
4. 按相关性从高到低排序
"""

def classify_comment(comment: str, index: int) -> list:
    """分类单条评论"""
    prompt = CLASSIFICATION_PROMPT.format(categories=categories_text, comment=comment)
    
    for i in range(3):
        try:
            response = llm_client.generate(prompt, temperature=0.1)
            response = response.replace('```json', '').replace('```', '').strip()
            data = json.loads(response)
            categories = data['categories']
            valid_categories = [cat for cat in categories if cat in my_categories]  # 过滤出在合法集合中的类别
            return valid_categories
        except Exception as e:
            print(f"索引为 {index} 的评论第 {i+1} 次尝试失败: {e}")
            if i < 2:
                time.sleep(1)
                continue
    
    print(f"索引为 {index} 的评论分类失败，已返回空列表")
    return []

In [12]:
# 批量分类
classification_results = []

for index, row in tqdm(df_raw.iterrows(), total=len(df_raw), desc="评论分类"):
    result = classify_comment(row['comment'], index)
    classification_results.append(result)
    time.sleep(0.1)

df_raw['categories'] = classification_results

评论分类:   0%|          | 0/2542 [00:00<?, ?it/s]

In [13]:
# 按小类聚合评论
category_comments = defaultdict(list)

for _, row in df_raw.iterrows():
    for cat in row['categories']:
        category_comments[cat].append({
            'comment': row['comment'],
            'quality_score': row['quality_score']
        })

print(f"共 {len(category_comments)} 个小类有评论:")
for cat, comments in sorted(category_comments.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"- {cat}: {len(comments)} 条评论")

共 14 个小类有评论:
- 整体满意度: 1855 条评论
- 前台服务: 1352 条评论
- 餐饮设施: 705 条评论
- 房间设施: 619 条评论
- 交通便利性: 465 条评论
- 卫生状况: 403 条评论
- 公共设施: 277 条评论
- 客房服务: 261 条评论
- 周边配套: 250 条评论
- 性价比: 156 条评论
- 安静程度: 147 条评论
- 退房/入住效率: 105 条评论
- 景观/朝向: 88 条评论
- 价格合理性: 13 条评论


In [14]:
# 保存扩展后的评论数据
enriched_file = PROCESSED_DATA_DIR / "enriched_comments.csv"
df_raw.to_csv(enriched_file)
print(f"扩展后的评论数据已保存: {enriched_file}")

# 保存过滤后的评论数据
df_filtered = df_raw[df_raw['quality_score'] >= 5]
filtered_file = PROCESSED_DATA_DIR / "filtered_comments.csv"
df_filtered.to_csv(filtered_file)
print(f"过滤后的评论数据已保存: {filtered_file}")

# 数据展示
df_filtered.head(1)

扩展后的评论数据已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\enriched_comments.csv
过滤后的评论数据已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\filtered_comments.csv


,comment,images,score,publish_date,room_type,fuzzy_room_type,travel_type,comment_len,log_comment_len,useful_count,log_useful_count,review_count,log_review_count,quality_score,categories
_id,,,,,,,,,,,,,,,
68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,"[ ""https://dimg04.c-ctrip.com/images/0230y1200...",5.0,2025-04-05,红棉大床套房,套房,家庭亲子,320,5.771441,0,0.0,7,2.079442,9,"[整体满意度, 餐饮设施, 前台服务]"


## 步骤3：类别摘要生成

为所需小类的评论生成关键词和摘要

In [15]:
# 摘要生成 Prompt
SUMMARIZATION_PROMPT = """
你是一位专业的内容摘要专家,请为以下类别的酒店评论生成一个摘要

类别: {category}
评论数量: {count}

部分评论内容:
{comments}

请生成:
1. 3-6个关键词（用逗号分隔）
2. 一个300-500字的摘要,涵盖该类别的主要观点和代表性内容,不要出现酒店的具体名称
3. 聚焦该类别下的话题，评论中出现的与该类别无关的内容请忽略

直接返回JSON格式:
{{
    "keywords": "关键词1,关键词2,关键词3,...",
    "summary": "摘要内容"
}}
"""

def generate_summary(category: str, comments: list) -> dict:
    """为一个类别生成摘要"""
    
    # 选择高质量评论作为样本
    sorted_comments = sorted(comments, key=lambda x: x['quality_score'], reverse=True)
    sample_comments = "\n".join([f"- {c['comment'][:1000]}" for c in sorted_comments[:500]])
    
    prompt = SUMMARIZATION_PROMPT.format(
        category=category,
        count=len(comments),
        comments=sample_comments
    )
    
    for i in range(3):
        try:
            response = llm_client.generate(prompt, temperature=0.5)
            response = response.replace('```json', '').replace('```', '').strip()
            data = json.loads(response)
            if data['keywords'] and data['summary']:
                return data
        except Exception as e:
            print(f"类别 {category} 的摘要第 {i+1} 次尝试失败: {e}")
            if i < 2:
                time.sleep(1)
                continue
    
    print(f"类别 {category} 摘要生成失败，已返回空字符串")
    return {'keywords': "", 'summary': ""}

In [16]:
# 批量生成摘要
summaries = []

for category, comments in tqdm(category_comments.items(), desc="生成摘要"):
    if category in my_categories:
        result = generate_summary(category, comments)
        summaries.append({
            'category': category,
            'keywords': result['keywords'],
            'summary': result['summary'],
            'comment_count': len(comments)
        })
        time.sleep(0.1)

print(f"已完成 {len([s for s in summaries if s['keywords']])} 个类别的摘要生成")

生成摘要:   0%|          | 0/14 [00:00<?, ?it/s]

已完成 14 个类别的摘要生成


In [17]:
# 保存摘要
summaries_file = PROCESSED_DATA_DIR / "category_summaries.json"
with open(summaries_file, 'w', encoding='utf-8') as f:
    json.dump(summaries, f, ensure_ascii=False, indent=2)

print(f"摘要已保存: {summaries_file}")

摘要已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\category_summaries.json


## 步骤4：反向 Query 生成

为每条高质量评论生成 1-3 个关联问题

In [18]:
# 反向 Query 生成 Prompt
REVERSE_QUERY_PROMPT = """
你是一位专业的问题生成专家,请基于以下酒店评论,生成1-3个用户可能会提出的问题,这些问题应该能被该评论很好地回答

评论内容: {comment}

要求:
1. 问题要自然、口语化,符合用户提问习惯
2. 问题要具体,能够被该评论直接回答
3. 问题要涵盖评论的核心信息点
4. 生成1-3个不同角度的问题
5. 若评论本身包含的信息量很少,则生成1-2个问题即可,不用强行生成3个冗余或无效的问题,需保证问题与评论的相关性和问题的多样性
6. 生成的问题中不要出现酒店的具体名称

直接返回JSON格式:
{{
    "queries": ["问题1", "问题2", "问题3"]
}}

注意: queries数组中包含1-3个问题
"""

def generate_reverse_queries(comment: str, index: int) -> list:
    """为单条评论生成反向 Query"""
    prompt = REVERSE_QUERY_PROMPT.format(comment=comment)
    
    for i in range(3):
        try:
            response = llm_client.generate(prompt, temperature=0.7)
            response = response.replace('```json', '').replace('```', '').strip()
            data = json.loads(response)
            queries = data['queries']
            if isinstance(queries, list):
                return queries
            else:
                raise TypeError(f"queries 数据类型错误: 期望 list, 实际为 {type(queries).__name__}")
        except Exception as e:
            print(f"索引为 {index} 的评论第 {i+1} 次尝试失败: {e}")
            if i < 2:
                time.sleep(1)
                continue
    
    print(f"索引为 {index} 的评论生成反向 Query 失败，已返回空列表")
    return []

In [19]:
# 批量生成反向 Query
all_queries = []

for index, row in tqdm(df_filtered.iterrows(), total=len(df_filtered), desc="生成反向Query"):
    queries = generate_reverse_queries(row['comment'], index)
    
    for query in queries:
        all_queries.append({
            'query': query,
            'comment_id': index,
            'comment': row['comment'],
            'room_type': row['room_type'],
            'fuzzy_room_type': row['fuzzy_room_type']
        })
    
    time.sleep(0.1)

df_queries = pd.DataFrame(all_queries)

print(f"总 Query 数: {len(df_queries)}")
print(f"平均每条评论: {len(df_queries) / len(df_filtered):.2f} 个 Query")

生成反向Query:   0%|          | 0/2171 [00:00<?, ?it/s]

总 Query 数: 6441
平均每条评论: 2.97 个 Query


In [20]:
# 保存反向 Query 数据集
queries_file = PROCESSED_DATA_DIR / "reverse_queries.csv"
df_queries.to_csv(queries_file, index=False)

print(f"反向 Query 数据已保存: {queries_file}")
df_queries.head(3)

反向 Query 数据已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\reverse_queries.csv


,query,comment_id,comment,room_type,fuzzy_room_type
0,这家酒店的装修和设施怎么样？看照片有点担心，实际体验好吗？,68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,红棉大床套房,套房
1,这家酒店的服务具体好在哪些地方？和其他五星级酒店比如何？,68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,红棉大床套房,套房
2,酒店的自助餐和餐饮水平如何？有什么特色菜推荐吗？,68027895e3c98b0941765706,房间非常好 装修很厚重奢华 一开始看评论 看酒店自己po的照片 感觉跟快捷酒店一样 有些害怕...,红棉大床套房,套房


## 步骤5：向量数据库构建

首先检查前四个步骤中 LLM 的指令遵循情况并手动调整不符合要求的数据

确认数据无误后，构建三个向量数据库：

1. **评论数据库** (DashVector) - 评论文本 embedding
2. **反向 Query 数据库** (DashVector) - Query 文本 embedding
3. **摘要数据库** (ChromaDB) - 类别关键词 embedding

In [21]:
# 重新加载数据
filtered_file = PROCESSED_DATA_DIR / "filtered_comments.csv"
queries_file = PROCESSED_DATA_DIR / "reverse_queries.csv"
summaries_file = PROCESSED_DATA_DIR / "category_summaries.json"

df_filtered = pd.read_csv(filtered_file, index_col=0)
df_queries = pd.read_csv(queries_file)
with open(summaries_file, 'r', encoding='utf-8') as f:
    summaries = json.load(f)

In [22]:
# DashVector Collection 创建/获取函数
def create_or_get_dashvector_collection(client, name: str, fields_schema: dict = None) -> any:
    """创建或获取 DashVector Collection"""
    collection = client.get(name)
    
    if collection is not None:
        stats = collection.stats()
        code = stats.code if hasattr(stats, "code") else -1
        
        if code == 0:
            print(f"Collection '{name}' 已存在")
            return collection
    
    # 创建新 Collection
    ret = client.create(
        name=name,
        dimension=EMBEDDING_DIMENSION,
        metric="cosine",
        dtype=float,
        fields_schema=fields_schema
    )
    
    if ret and hasattr(ret, "code") and ret.code == 0:
        print(f"Collection '{name}' 创建成功")
    else:
        raise RuntimeError(f"创建 Collection 失败: {ret}")
    
    return client.get(name)

### 5.1 构建评论数据库（DashVector）

In [23]:
# 创建评论 Collection
comment_fields_schema = {
    'comment': str,
    'room_type': str,
    'fuzzy_room_type': str
}
comment_collection = create_or_get_dashvector_collection(dashvector_client, "comment_database", comment_fields_schema)

Collection 'comment_database' 创建成功


In [24]:
# 生成评论 embedding 并插入
comment_texts = df_filtered['comment'].tolist()
total_inserted = 0

for i in tqdm(range(0, len(comment_texts), BATCH_SIZE), desc="构建评论数据库"):
    batch_texts = comment_texts[i: i + BATCH_SIZE]
    batch_embeddings = embedding_client.embed_batch(batch_texts)
    
    # 构建文档
    docs = []
    for j, embedding in enumerate(batch_embeddings):
        idx = i + j
        row = df_filtered.iloc[idx]
        
        docs.append(Doc(
            id=row.name,
            vector=embedding,
            fields={
                'comment': row['comment'],
                'room_type': row['room_type'],
                'fuzzy_room_type': row['fuzzy_room_type']
            }
        ))
    
    # 插入
    response = comment_collection.upsert(docs)
    if hasattr(response, "code") and response.code == 0:
        total_inserted += len(docs)
    
    time.sleep(0.5)

print(f"评论数据库构建完成, 共更新 {total_inserted} 条记录")

构建评论数据库:   0%|          | 0/218 [00:00<?, ?it/s]

评论数据库构建完成, 共更新 2171 条记录


### 5.2 构建反向 Query 数据库（DashVector）

In [25]:
# 创建反向 Query Collection
query_fields_schema = {
    'query': str,
    'comment_id': str,
    'comment': str,
    'room_type': str,
    'fuzzy_room_type': str
}
query_collection = create_or_get_dashvector_collection(dashvector_client, "reverse_query_database", query_fields_schema)

Collection 'reverse_query_database' 创建成功


In [26]:
# 生成 Query embedding 并插入
query_texts = df_queries['query'].tolist()
total_inserted = 0

for i in tqdm(range(0, len(query_texts), BATCH_SIZE), desc="构建反向Query数据库"):
    batch_texts = query_texts[i: i + BATCH_SIZE]
    batch_embeddings = embedding_client.embed_batch(batch_texts)
    
    docs = []
    for j, embedding in enumerate(batch_embeddings):
        idx = i + j
        row = df_queries.iloc[idx]
        
        docs.append(Doc(
            id=f'query_{idx}',
            vector=embedding,
            fields={
                'query': row['query'],
                'comment_id': row['comment_id'],
                'comment': row['comment'],
                'room_type': row['room_type'],
                'fuzzy_room_type': row['fuzzy_room_type']
            }
        ))

    # 插入
    response = query_collection.upsert(docs)
    if hasattr(response, "code") and response.code == 0:
        total_inserted += len(docs)
    
    time.sleep(0.5)

print(f"反向 Query 数据库构建完成, 共更新 {total_inserted} 条记录")

构建反向Query数据库:   0%|          | 0/645 [00:00<?, ?it/s]

反向 Query 数据库构建完成, 共更新 6441 条记录


### 5.3 构建摘要数据库（ChromaDB）

In [27]:
# 删除旧数据库
try:
    chroma_client.delete_collection("summary_database")
    print("删除旧的摘要数据库")
except:
    pass

# 创建新数据库
summary_collection = chroma_client.create_collection(
    name="summary_database",
    metadata={'description': "摘要数据库: 基于类别关键词的向量检索", 'hnsw:space': "cosine"}  # 使用余弦相似度，若不指定，默认为 L2 欧氏距离
)

print("摘要数据库创建成功")

摘要数据库创建成功


In [28]:
# 生成关键词 Embedding 并插入
keywords_list = [s['keywords'] for s in summaries]

for i in tqdm(range(0, len(keywords_list), BATCH_SIZE), desc="构建摘要数据库"):
    batch_texts = keywords_list[i: i + BATCH_SIZE]
    batch_embeddings = embedding_client.embed_batch(batch_texts)

    num = len(batch_embeddings)
    summary_collection.add(
        ids=[f"summary_{j}" for j in range(i, i + num)],
        embeddings=batch_embeddings,
        documents=[s['summary'] for s in summaries[i: i + num]],
        metadatas=[
            {
                'category': s['category'],
                'keywords': s['keywords'],
                'comment_count': s['comment_count']
            }
            for s in summaries[i: i + num]
        ]
    )

print(f"摘要数据库构建完成, 共更新 {len(summaries)} 条记录")

构建摘要数据库:   0%|          | 0/2 [00:00<?, ?it/s]

摘要数据库构建完成, 共更新 14 条记录


## 向量数据库测试

In [29]:
# 验证评论数据库
test_query = "酒店博物馆如何？"
test_embedding = embedding_client.embed_batch([test_query])[0]

results = comment_collection.query(vector=test_embedding, topk=3)

print("评论数据库查询测试:")
print(f"- 查询: {test_query}")
print(f"- 返回结果数: {len(results)}")

print(f"\nTop 1 结果:")
print(f"- 相似度: {results[0].score:.4f}")
print(f"- 评论: {results[0].fields.get('comment', '')}")
print(f"- 房型: {results[0].fields.get('room_type', '')}")

评论数据库查询测试:
- 查询: 酒店博物馆如何？
- 返回结果数: 3

Top 1 结果:
- 相似度: 0.3393
- 评论: 酒店本身就是一个景点，最惊艳的是四楼的博物馆！
- 房型: 花园双床房


In [30]:
# 验证反向 Query 数据库
results = query_collection.query(vector=test_embedding, topk=3)

print("反向 Query 数据库查询测试:")
print(f"- 查询: {test_query}")
print(f"- 返回结果数: {len(results)}")

print(f"\nTop 1 结果:")
print(f"- 相似度: {results[0].score:.4f}")
print(f"- Query: {results[0].fields.get('query', '')}")
print(f"- 关联评论: {results[0].fields.get('comment', '')}")
print(f"- 模糊房型: {results[0].fields.get('fuzzy_room_type', '')}")

反向 Query 数据库查询测试:
- 查询: 酒店博物馆如何？
- 返回结果数: 3

Top 1 结果:
- 相似度: 0.1171
- Query: 酒店博物馆值得去看吗？对住客有没有什么特别待遇？
- 关联评论: 老品牌，颇有底蕴的酒店。给人富丽贵气的感觉。喜欢酒店礼宾部Gary小陈的介绍。相比之下，四楼博物馆门口小哥的态度很让人费解，一个酒店博物馆不就是为了推介吸引参观吗，什么要预约扫码，还不对住客有专享，你以为自己专属文旅就可以这种态度对宾客？搞笑了，让人没胃口看了。
- 模糊房型: 双床房


In [31]:
# 验证摘要数据库
results = summary_collection.query(query_embeddings=[test_embedding], n_results=1)

print("摘要数据库查询测试:")
print(f"- 查询: {test_query}")
print(f"- 返回结果数: {len(results['ids'][0])}")

# 获取 Top 1 结果
metadata = results['metadatas'][0][0]
document = results['documents'][0][0]

print(f"\nTop 1 结果:")
print(f"- 类别: {metadata.get('category', '')}")
print(f"- 关键词: {metadata.get('keywords', '')}")
print(f"- 摘要: {document[:200]}...")

摘要数据库查询测试:
- 查询: 酒店博物馆如何？
- 返回结果数: 1

Top 1 结果:
- 类别: 公共设施
- 关键词: 岭南园林花园,瀑布与锦鲤,酒店博物馆,金碧辉煌大堂,旋转楼梯与壁画,导览讲解服务
- 摘要: 关于酒店的公共设施，评论普遍给予极高评价，认为其是酒店最核心的亮点与竞争力。主要观点集中在以下几个方面：

首先，酒店内独具特色的岭南园林花园是几乎所有评论都提及的焦点。住客们惊叹于在市中心竟能拥有如此精致、静谧且打理用心的花园，其中人工双瀑布、小桥流水、池塘与成群锦鲤构成了核心景观，被誉为“城市绿洲”和“世外桃源”，特别适合亲子游玩、散步与拍照，完美诠释了酒店名称。

其次，酒店内部充满艺术与文...


## 步骤6：倒排索引构建

基于 BM25 算法的传统关键词检索，支持精确匹配召回

In [32]:
# 倒排索引类定义
class InvertedIndex:
    """基于 BM25 的倒排索引"""
    
    def __init__(self, k1: float = 1.5, b: float = 0.75, stopwords_file: str = None):
        """
        参数:
            k1: BM25 参数，控制词频饱和度
            b: BM25 参数，控制文档长度归一化程度
        """
        self.k1 = k1
        self.b = b
        self.index = {}          # {term: {doc_id: term_freq}}
        self.doc_lengths = {}    # {doc_id: doc_length}
        self.avg_doc_length = 0
        self.num_docs = 0
        self.documents = {}      # {doc_id: document_content}

        # 加载停用词
        self.stopwords = set()
        if stopwords_file and Path(stopwords_file).exists():
            with open(stopwords_file, encoding='utf-8') as f:
                self.stopwords.update([line.strip() for line in f])
            try:
                self.stopwords.update(nltk.corpus.stopwords.words('english'))  # 加载英文停用词
            except:
                print("警告: 未能加载 NLTK 英文停用词")
        
        # 字典预加载
        jieba.initialize()
    
    def tokenize(self, text: str) -> list[str]:
        """分词与过滤"""     
        
        # 删除空白字符
        text = re.sub(r'\s+', '', text)
        
        # 中文分词
        tokens = jieba.lcut(text)
        
        # 过滤停用词、非中英文字符，统一小写
        pattern = re.compile(r'[^\u4e00-\u9fffa-zA-Z]')
        tokens = [token.lower() for token in tokens if token.lower() not in self.stopwords and not pattern.search(token)]
        
        return tokens
    
    def build(self, documents: dict[str, str]):
        """
        构建倒排索引
        
        参数:
            documents: {doc_id: document_text}
        """
        self.documents = documents
        self.num_docs = len(documents)
        
        # 统计文档长度
        total_length = 0
        for doc_id, text in tqdm(documents.items(), desc="分词与统计"):
            tokens = self.tokenize(text)
            doc_length = len(tokens)
            self.doc_lengths[doc_id] = doc_length
            total_length += doc_length
            
            # 构建倒排索引
            term_freq = Counter(tokens)
            for term, freq in term_freq.items():
                if term not in self.index:
                    self.index[term] = {}
                self.index[term][doc_id] = freq
        
        self.avg_doc_length = total_length / self.num_docs if self.num_docs > 0 else 0
        print(f"倒排索引构建完成: {len(self.index)} 个词项, {self.num_docs} 篇文档")
        print(f"平均文档长度: {self.avg_doc_length:.2f} 个词")
    
    def search(self, query: str, topk: int = 10) -> list[tuple[str, float]]:
        """
        BM25 检索
        
        参数:
            query: 查询文本
            topk: 返回 Top-K 结果
        
        返回:
            [(doc_id, bm25_score), ...]
        """
        query_tokens = self.tokenize(query)

        if not query_tokens:
            return []
        
        # 计算 IDF
        idf = {}
        for term in query_tokens:
            if term in self.index:
                df = len(self.index[term])  # 文档频率
                idf[term] = max(0, (self.num_docs - df + 0.5) / (df + 0.5) + 1)
        
        # 计算 BM25 分数
        scores = {}
        for term in query_tokens:
            if term not in self.index:
                continue
            
            for doc_id, tf in self.index[term].items():
                if doc_id not in scores:
                    scores[doc_id] = 0
                
                doc_length = self.doc_lengths[doc_id]
                norm_factor = 1 - self.b + self.b * (doc_length / self.avg_doc_length)
                term_score = idf[term] * (tf * (self.k1 + 1)) / (tf + self.k1 * norm_factor)
                scores[doc_id] = scores.get(doc_id, 0) + term_score
        
        # 排序并返回 Top-K
        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topk]
        return sorted_docs
    
    def save(self, filepath: str):
        """保存索引到文件"""
        with open(filepath, 'wb') as f:
            pickle.dump({
                'index': self.index,
                'doc_lengths': self.doc_lengths,
                'avg_doc_length': self.avg_doc_length,
                'num_docs': self.num_docs,
                'documents': self.documents,
                'k1': self.k1,
                'b': self.b,
                'stopwords': self.stopwords
            }, f)
        print(f"倒排索引已保存: {filepath}")
    
    def load(self, filepath: str):
        """从文件加载索引"""
        with open(filepath, 'rb') as f:
            data = pickle.load(f)
            self.index = data['index']
            self.doc_lengths = data['doc_lengths']
            self.avg_doc_length = data['avg_doc_length']
            self.num_docs = data['num_docs']
            self.documents = data['documents']
            self.k1 = data['k1']
            self.b = data['b']
            self.stopwords = data.get('stopwords', set())
        print(f"倒排索引已加载")

In [33]:
# 构建倒排索引
stopwords_file = CONFIG_DIR / "stopwords_chinese.txt"                          # 中文停用词文件路径
inverted_index = InvertedIndex(k1=1.5, b=0.75, stopwords_file=stopwords_file)
documents = {idx: row['comment'] for idx, row in df_filtered.iterrows()}       # 使用 comment_id 作为 doc_id
inverted_index.build(documents)

# 保存倒排索引
index_file = PROCESSED_DATA_DIR / "inverted_index.pkl"
inverted_index.save(index_file)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\22418\AppData\Local\Temp\jieba.cache
Loading model cost 0.605 seconds.
Prefix dict has been built successfully.


分词与统计:   0%|          | 0/2171 [00:00<?, ?it/s]

倒排索引构建完成: 10734 个词项, 2171 篇文档
平均文档长度: 41.62 个词
倒排索引已保存: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\inverted_index.pkl


## 倒排索引测试

In [34]:
# 加载倒排索引
inverted_index = InvertedIndex()
inverted_index.load(index_file)

# 验证倒排索引
test_query_bm25 = "酒店博物馆和瀑布花园如何？"
bm25_results = inverted_index.search(test_query_bm25, topk=3)

print("\n倒排索引查询测试:")
print(f"- 原始查询: {test_query_bm25}")
print(f"- 分词结果: {inverted_index.tokenize(test_query_bm25)}")
print(f"- 返回结果数: {len(bm25_results)}")

# 获取 Top 1 结果
doc_id, score = bm25_results[0]
comment = df_filtered.loc[doc_id]

print(f"\nTop 1 结果:")
print(f"- BM25 分数: {score:.4f}")
print(f"- 评论: {comment['comment']}")

倒排索引已加载

倒排索引查询测试:
- 原始查询: 酒店博物馆和瀑布花园如何？
- 分词结果: ['酒店', '博物馆', '瀑布', '花园']
- 返回结果数: 3

Top 1 结果:
- BM25 分数: 26.2280
- 评论: 酒店就是住在花园里，亮点是花园瀑布下午茶，还有4楼博物馆也可免费参观，很不错的体验哦


## 总结

第一模块（离线知识库构建）已全部完成

In [35]:
print(f"数据过滤:")
print(f"- 原数据评论数: {len(df_raw)}")
print(f"- 过滤后评论数: {len(df_filtered)} (quality_score >= 5)")

print(f"\n向量数据库:")
print(f"1. 评论数据库 (DashVector): {len(df_filtered)} 条")
print(f"2. 反向 Query 数据库 (DashVector): {len(df_queries)} 条")
print(f"3. 摘要数据库 (ChromaDB): {len(summaries)} 条")

print(f"\n倒排索引:")
print(f"4. BM25 倒排索引: {len(inverted_index.index)} 个词项, {inverted_index.num_docs} 条评论")

print(f"\n输出文件:")
print(f"- {enriched_file}")
print(f"- {filtered_file}")
print(f"- {queries_file}")
print(f"- {summaries_file}")
print(f"- {chroma_db_path}\\ (ChromaDB)")
print(f"- {index_file}")

数据过滤:
- 原数据评论数: 2542
- 过滤后评论数: 2171 (quality_score >= 5)

向量数据库:
1. 评论数据库 (DashVector): 2171 条
2. 反向 Query 数据库 (DashVector): 6441 条
3. 摘要数据库 (ChromaDB): 14 条

倒排索引:
4. BM25 倒排索引: 10734 个词项, 2171 条评论

输出文件:
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\enriched_comments.csv
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\filtered_comments.csv
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\reverse_queries.csv
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\category_summaries.json
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\chroma_db\ (ChromaDB)
- C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论\data\processed\inverted_index.pkl
